In [14]:
import pandas as pd
import glob
import requests
import json
import time

# Project notes

- Identify transactions that are missing from OC that are on any of the accounts (meaning banks, stripe, wise, paypal)
- Identify any transactions on the platform that are missing from the accounts
- Prep a balance/p&l with minimal wrangling

Project steps:

- Clarify project intention and goal (fill in project assessment in Make OC)
- Map out data 
- Write scripts 
- Analyze and prepare accounts

# Load config

In [15]:
# host variables
host = 'opensource'
dateFrom = '2022-12-31T22:59:59.999Z'
dateTo = '2023-12-31T22:59:59.999Z'
year = '2023'

In [16]:
# load API tokens from .env file
import os
from dotenv import load_dotenv
load_dotenv()
octoken = os.getenv('OCTOKEN')
wisetoken = os.getenv('WISETOKEN')

# set the profile id of the target Wise profile
wiseprofile = os.getenv('WISEPROFILE')

In [17]:
# Reload the data or not - set these to false after the first run to avoid reloading
reloadOC = False
reloadWise = False
reloadStripe = False

# Get data

TODO: Query below returns null for all merchandId. Need to get merchantId for all transactions where available to reconcile with Stripe, Wise and Paypal.

In [18]:
# load transactions from open collective or from local file

def fetch_transactions():
    url = 'https://opencollective.com/api/graphql/v2'

    # Your GraphQL query
    query = """
    query TransactionsTable(
        $hostAccount: AccountReferenceInput,
        $limit: Int!,
        $offset: Int!,
        $type: TransactionType,
        $paymentMethodType: [PaymentMethodType],
        $dateFrom: DateTime,
        $dateTo: DateTime,
        $kind: [TransactionKind],
        $includeIncognitoTransactions: Boolean,
        $includeGiftCardTransactions: Boolean,
        $includeChildrenTransactions: Boolean,
        $virtualCard: [VirtualCardReferenceInput],
        $orderBy: ChronologicalOrderInput,
        $group: String,
        $includeHost: Boolean,
        $expense: ExpenseReferenceInput,
        $order: OrderReferenceInput
      ) {
        transactions(
          host: $hostAccount
          limit: $limit
          offset: $offset
          type: $type
          paymentMethodType: $paymentMethodType
          dateFrom: $dateFrom
          dateTo: $dateTo
          kind: $kind
          includeIncognitoTransactions: $includeIncognitoTransactions
          includeGiftCardTransactions: $includeGiftCardTransactions
          includeChildrenTransactions: $includeChildrenTransactions
          includeDebts: true
          virtualCard: $virtualCard
          orderBy: $orderBy
          group: $group
          includeHost: $includeHost
          expense: $expense
          order: $order
        ) {
          ...TransactionsTableQueryCollectionFragment
          __typename
        }
      }
      
      fragment TransactionsTableQueryCollectionFragment on TransactionCollection {
        totalCount
        offset
        limit
        nodes {
          id
          uuid
          kind
          amount {
            currency
            valueInCents
            __typename
          }
          amountInHostCurrency {
            currency
            valueInCents
            __typename
          }
          netAmount {
            currency
            valueInCents
            __typename
          }
          netAmountInHostCurrency {
            currency
            valueInCents
            __typename
          }
          paymentProcessorFee {
            currency
            valueInCents
            __typename
          }
          paymentMethod {
            name
            service
            sourcePaymentMethod {
              id
              name
              service
              __typename
            }
            type
          }
          giftCardEmitterAccount {  
            id
            name
            slug
            __typename
          }
          group
          type
          description
          createdAt
          merchantId
          isRefunded
          isRefund
          refundTransaction {
            id
          }
          isOrderRejected
          account {
            ... on AccountWithParent {
              parent {
                id
                slug
                name
              }
            }
            id
            legacyId
            name
            slug
            isIncognito
            type
            __typename
          }
          oppositeAccount {
            id
            legacyId
            name
            slug
            isIncognito
            type
            __typename
          }
          expense {
            id
            type
            description
            invoiceInfo 
            tags
            payee {
              id
              name
              slug
              type
              __typename
            }
            __typename
          }
          permissions {
            id
            canRefund
            canDownloadInvoice
            canReject
            __typename
          }
          __typename
        }
        __typename
      }
    """

    headers = {
        "Authorization": f"Bearer {octoken}",
        "Content-Type": "application/json"
    }

    # Initial variables setup
    variables = {
        "hostAccount": {"slug": host},
        "includeIncognitoTransactions": True,
        "includeChildrenTransactions": True,
        "limit": 10000,  # Adjust if necessary but keep a sensible number to avoid server strain
        "offset": 0,  # Will be adjusted for each subsequent request
        "dateFrom": dateFrom,
        "dateTo": dateTo,
        "orderBy": {"field": "CREATED_AT", "direction": "DESC"},
        "includeHost": True
    }

    all_transactions = []  # To hold all transactions
    while True:
        # Make the HTTP request
        response = requests.post(
            url, json={'query': query, 'variables': variables}, headers=headers)
        json_data = response.json()

        # Error handling: Ensure to include proper error handling here
        if 'errors' in json_data:
            print(json.dumps(json_data['errors']))
            break

        # Extract data
        transactions = json_data['data']['transactions']['nodes']
        all_transactions.extend(transactions)

        # Pagination: Update offset
        variables['offset'] += variables['limit']

        # Check if all transactions are fetched
        if len(all_transactions) >= json_data['data']['transactions']['totalCount']:
            break
        
        # print progress
        print(f'Fetched {len(all_transactions)} transactions')
        # sleep for 10 seconds to avoid server strain
        time.sleep(1)

    return all_transactions

# if account has property parent, replace account with parent
def replace_account_with_parent(transaction):
  if 'parent' in transaction['account']:
    transaction['account'] = transaction['account']['parent']
  return transaction

# apply the function to all transactions
def post_process_transactions(all_transactions):
  return list(map(replace_account_with_parent, all_transactions))

if reloadOC:
  all_transactions = fetch_transactions()
  # dump all transactions to a json file
  post_process_transactions(all_transactions)
  with open(f'data/{host}/{year}/{host}_{year}_all_platform_transactions.json', 'w') as f:
      json.dump(all_transactions, f, indent=2)
  # convert the json file to a dataframe
  df_platform_transactions = pd.json_normalize(all_transactions)
  # save dataframe as pickle
  df_platform_transactions.to_pickle(f'data/{host}/{year}/df_{host}_{year}_all_platform_transactions.pkl')
else:
  df_platform_transactions = pd.read_pickle(f'data/{host}/{year}/df_{host}_{year}_all_platform_transactions.pkl')


In [19]:
# load transactions from wise

# Define the base URL for the Wise API
BASE_URL = 'https://api.transferwise.com/v1/transfers'

def fetch_all_transfers(profile_id=None, status=None, source_currency=None, 
                        target_currency=None, created_date_start=None, 
                        created_date_end=None, limit=100):
    headers = {
        'Authorization': f'Bearer {wisetoken}'
    }
    offset = 0
    all_transfers = []

    while True:
        # Prepare query parameters
        query_params = {'limit': limit, 'offset': offset}
        if profile_id:
            query_params['profile'] = profile_id
        if status:
            query_params['status'] = status
        if source_currency:
            query_params['sourceCurrency'] = source_currency
        if target_currency:
            query_params['targetCurrency'] = target_currency
        if created_date_start:
            query_params['createdDateStart'] = created_date_start
        if created_date_end:
            query_params['createdDateEnd'] = created_date_end

        # Make the GET request to the Wise API
        response = requests.get(BASE_URL, headers=headers, params=query_params)
        data = response.json()

        print(data)

        # Break if there are no more results
        if not data:
            break

        for transfer in data:
            all_transfers.append({
                'id': transfer['id'],
                'user': transfer['user'],
                'targetAccount': transfer['targetAccount'],
                'sourceAccount': transfer['sourceAccount'],
                'quote': transfer['quote'],
                'quoteUuid': transfer['quoteUuid'],
                'status': transfer['status'],
                'reference': transfer['reference'],
                'rate': transfer['rate'],
                'created': transfer['created'],
                'business': transfer['business'],
                'transferRequest': transfer['transferRequest'],
                'details.reference': transfer.get('details', {}).get('reference'),
                'hasActiveIssues': transfer['hasActiveIssues'],
                'sourceCurrency': transfer['sourceCurrency'],
                'sourceValue': transfer['sourceValue'],
                'targetCurrency': transfer['targetCurrency'],
                'targetValue': transfer['targetValue'],
                'customerTransactionId': transfer['customerTransactionId']
            })

        # Update the offset for the next iteration
        offset += limit

    return all_transfers

if reloadWise:
    transfers = fetch_all_transfers(profile_id=wiseprofile, created_date_start=dateFrom, created_date_end=dateTo)

    # create transfers dataframe
    df_wise_transactions = pd.DataFrame()

    for transfer in transfers:
        # add to dataframe with concat
        df_wise_transactions = pd.concat([df_wise_transactions, pd.DataFrame([transfer])])

    # save dataframe as pickle
    df_wise_transactions.to_pickle(f'data/{host}/{year}/df_{host}_{year}_wise_transactions.pkl')

In [20]:
# load transactions from stripe export
if reloadStripe:
    df_stripe_in = pd.read_csv(f'data/{host}/{year}/{host}_{year}_stripe.csv')
    # ensure only relevant columns
    df_stripe_in = df_stripe_in[[
    'id',
    'Description',
    'Seller Message',
    'Amount',
    'Amount Refunded',
    'Currency',
    'Converted Amount',
    'Converted Amount Refunded',
    'Fee',
    'Taxes On Fee',
    'Converted Currency',
    'Mode',
    'Status',
    'Statement Descriptor',
    'Customer ID',
    'Customer Description',
    'Captured',
    'Card ID',
    'PaymentIntent ID',
    'Application Fee',
    'Application ID',
    'from (metadata)',
    'to (metadata)'
    ]]
    df_stripe_in.to_pickle(f'data/{host}/{year}/df_{host}_{year}_stripe_transactions.pkl')
    


In [21]:
# load open collective data
df_platform = pd.read_pickle(f'data/{host}/{year}/df_{host}_{year}_all_platform_transactions.pkl')

# load stripe data
df_stripe = pd.read_pickle(f'data/{host}/{year}/df_{host}_{year}_stripe_transactions.pkl')

# load wise data
df_wise = pd.read_pickle(f'data/{host}/{year}/df_{host}_{year}_wise_transactions.pkl')

# load bank data
dir = f'data/{host}/{year}/oscbank_1'
df_bank_1 = pd.concat([pd.read_csv(f) for f in glob.glob(f'{dir}/*.csv')], ignore_index=True)
df_bank_1 = df_bank_1.dropna(axis=1, how='all')

dir = f'data/{host}/{year}/oscbank_2'
df_bank_2 = pd.concat([pd.read_csv(f) for f in glob.glob(f'{dir}/*.csv')], ignore_index=True)
df_bank_2 = df_bank_2.dropna(axis=1, how='all')

# Post process data

In [22]:
# post process

# remove failed and pending transactions from Stripe
df_stripe = df_stripe[df_stripe['Status'] != 'Failed']
df_stripe = df_stripe[df_stripe['Status'] != 'Pending']

# remove cancelled from Wise
df_wise = df_wise[df_wise['status'] != 'cancelled']

# Inspect transactions dataframe

In [23]:
# show all columns
pd.set_option('display.max_columns', None)
df_platform.sample(10)

,id,uuid,kind,paymentMethod,giftCardEmitterAccount,group,type,description,createdAt,merchantId,isRefunded,isRefund,refundTransaction,isOrderRejected,expense,__typename,amount.currency,amount.valueInCents,amount.__typename,amountInHostCurrency.currency,amountInHostCurrency.valueInCents,amountInHostCurrency.__typename,netAmount.currency,netAmount.valueInCents,netAmount.__typename,netAmountInHostCurrency.currency,netAmountInHostCurrency.valueInCents,netAmountInHostCurrency.__typename,paymentProcessorFee.currency,paymentProcessorFee.valueInCents,paymentProcessorFee.__typename,account.id,account.legacyId,account.name,account.slug,account.isIncognito,account.type,account.__typename,oppositeAccount.id,oppositeAccount.legacyId,oppositeAccount.name,oppositeAccount.slug,oppositeAccount.isIncognito,oppositeAccount.type,oppositeAccount.__typename,permissions.id,permissions.canRefund,permissions.canDownloadInvoice,permissions.canReject,permissions.__typename,paymentMethod.name,paymentMethod.service,paymentMethod.sourcePaymentMethod,paymentMethod.type,expense.id,expense.type,expense.description,expense.invoiceInfo,expense.tags,expense.payee.id,expense.payee.name,expense.payee.slug,expense.payee.type,expense.payee.__typename,expense.__typename,refundTransaction.id,giftCardEmitterAccount.id,giftCardEmitterAccount.name,giftCardEmitterAccount.slug,giftCardEmitterAccount.__typename
28538,25202702-373b-4622-94a7-d3966b21d3c6,25202702-373b-4622-94a7-d3966b21d3c6,CONTRIBUTION,NaN,NaN,67abae6a-ffb9-42be-a996-8d343356150f,CREDIT,Financial contribution to Cider Collective (On...,2023-01-02T21:29:37.258Z,None,False,False,NaN,False,NaN,Credit,USD,250,Amount,USD,250,Amount,USD,189,Amount,USD,189,Amount,USD,-61,Amount,eng0kzdy-vor4pzbr-yorpbma8-37xlw95j,464673.0,Cider Collective,ciderapp,False,COLLECTIVE,Collective,vedj9wro-z3a56d3z-8w5p7blg-8x4m0ykn,592606,Major Dominik,major-dominik,False,INDIVIDUAL,Individual,vrgbk357-l4x96e00-l4aqomew-a0jdyzn8,False,False,False,TransactionPermissions,None,PAYPAL,NaN,PAYMENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61402,a793a54e-987b-469a-9718-f5d3c3bfcdb8,a793a54e-987b-469a-9718-f5d3c3bfcdb8,HOST_FEE_SHARE,NaN,NaN,a743812c-210e-456c-9bab-164cf5b493b1,DEBIT,Host Fee Share,2023-01-01T02:10:21.744Z,None,False,False,NaN,False,NaN,Debit,USD,-50,Amount,USD,-50,Amount,USD,-50,Amount,USD,-50,Amount,USD,0,Amount,8a47byg9-nxozdp80-xm6mjlv0-3rek5w8k,11004.0,Open Source Collective,opensource,False,ORGANIZATION,Organization,lk9mbw7y-48r3zq3w-g860ej5l-avnodgx7,8686,Open Collective,opencollective,False,ORGANIZATION,Organization,8rzownxl-9e50pxka-zvjqymvb-dgk7j43a,False,False,False,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13182,b96ecbb4-e258-46fd-900b-304f6efd1b01,b96ecbb4-e258-46fd-900b-304f6efd1b01,HOST_FEE,NaN,NaN,42afe396-399a-4148-9e8c-c6297322b0b4,CREDIT,Host Fee,2023-01-18T10:11:24.058Z,None,False,False,NaN,False,NaN,Credit,USD,20,Amount,USD,20,Amount,USD,20,Amount,USD,20,Amount,USD,0,Amount,8a47byg9-nxozdp80-xm6mjlv0-3rek5w8k,11004.0,Open Source Collective,opensource,False,ORGANIZATION,Organization,ov349mrw-gz75lpyy-4npa08dj-eybknoxd,10350,JHipster,generator-jhipster,False,COLLECTIVE,Collective,a47byg9n-xozdp8wo-x7mpmjlv-03rek5w8,False,False,False,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51253,7d178418-8548-446d-b0a2-256010140712,7d178418-8548-446d-b0a2-256010140712,HOST_FEE,NaN,NaN,8fde15c8-bf8d-4ae7-94cf-63726641627c,CREDIT,Host Fee,2023-01-01T12:05:10.998Z,None,False,False,NaN,False,NaN,Credit,USD,150,Amount,USD,150,Amount,USD,150,Amount,USD,150,Amount,USD,0,Amount,8a47byg9-nxozdp80-xm6mjlv0-3rek5w8k,11004.0,Open Source Collective,opensource,False,ORGANIZATION,Organization,88rzownx-l9e50pxm-e8w6ymvb-dgk7j43a,173863,FlyByWire Simulations,flybywire,False,COLLECTIVE,Collective,a47byg9n-xozdp8wg-ovbpmjlv-03rek5w8,False,False,False,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [96]:
# count transactions by kind
df_platform.groupby(['kind']).size()

kind
ADDED_FUNDS                        216
BALANCE_TRANSFER                     4
CONTRIBUTION                     15092
EXPENSE                            608
HOST_FEE                         30040
HOST_FEE_SHARE                   15020
HOST_FEE_SHARE_DEBT               4432
PAYMENT_PROCESSOR_COVER             82
PAYMENT_PROCESSOR_DISPUTE_FEE        1
PREPAID_PAYMENT_METHOD               1
dtype: int64

# Reconciling Stripe

TODO: Before we can do this, we need to get the merchantIDs.

In [97]:
# copy df_strip to df_s
df_s = df_stripe.copy()

In [98]:
df_s.head()

,id,Description,Seller Message,Amount,Amount Refunded,Currency,Converted Amount,Converted Amount Refunded,Fee,Taxes On Fee,Converted Currency,Mode,Status,Statement Descriptor,Customer ID,Customer Description,Captured,Card ID,PaymentIntent ID,Application Fee,Application ID,from (metadata),to (metadata)
0,ch_3OTXgHBYycQg1OMf0IfKN8Sw,Monthly financial contribution to Logseq (Back...,Payment complete.,5.00,0.0,usd,5.00,0.0,0.77,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,cus_PI7wZ2sLFYmMPi,https://opencollective.com/sanket-sharma,True,pm_1OTXmVBYycQg1OMfzXUwCvAm,pi_3OTXgHBYycQg1OMf0PkaNI3M,0.25,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,https://opencollective.com/sanket-sharma,https://opencollective.com/logseq
3,ch_3OTXVMBYycQg1OMf0qVZ5tjm,Financial contribution to OpenSCAD,Payment complete.,50.00,0.0,usd,50.00,0.0,4.25,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OTXVpBYycQg1OMfien2j8Cr,pi_3OTXVMBYycQg1OMf02f4Vhyl,2.50,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/openscad
4,ch_3OTULSBYycQg1OMf10wlDceK,Monthly financial contribution to LibreELEC,Payment complete.,10.00,0.0,usd,10.00,0.0,1.24,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,cus_PI4UZkic5DVDO1,https://opencollective.com/daniel-dolejska,True,pm_1OTULSBYycQg1OMf7Lt0BwEi,pi_3OTULSBYycQg1OMf1saV7WWG,0.50,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,https://opencollective.com/daniel-dolejska,https://opencollective.com/libreelec
5,ch_3OTUG8BYycQg1OMf0FRuZULZ,Financial contribution to Spotube,Payment complete.,5.00,0.0,usd,5.00,0.0,0.77,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OTUI5BYycQg1OMfAbkDmXhS,pi_3OTUG8BYycQg1OMf0AcGFa2r,0.25,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/spotube
7,ch_3OTT8EBYycQg1OMf0gR3PzrY,Financial contribution to .fmbot (.fmbot lifet...,Payment complete.,49.99,0.0,usd,49.99,0.0,4.25,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OTT94BYycQg1OMf9AOHyAVx,pi_3OTT8EBYycQg1OMf0xdn31JV,2.50,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/fmbot


In [99]:
# df_platform where kind is CONTRIBUTION and paymentMethod.service is STRIPE
df_p = df_platform[(df_platform['kind'] == 'CONTRIBUTION') & (df_platform['paymentMethod.service'] == 'STRIPE')]

In [100]:
df_p.head()

,id,uuid,kind,paymentMethod,giftCardEmitterAccount,group,type,description,createdAt,merchantId,isRefunded,isRefund,refundTransaction,isOrderRejected,expense,__typename,amount.currency,amount.valueInCents,amount.__typename,amountInHostCurrency.currency,amountInHostCurrency.valueInCents,amountInHostCurrency.__typename,netAmount.currency,netAmount.valueInCents,netAmount.__typename,netAmountInHostCurrency.currency,netAmountInHostCurrency.valueInCents,netAmountInHostCurrency.__typename,paymentProcessorFee.currency,paymentProcessorFee.valueInCents,paymentProcessorFee.__typename,account.id,account.legacyId,account.name,account.slug,account.isIncognito,account.type,account.__typename,oppositeAccount.id,oppositeAccount.legacyId,oppositeAccount.name,oppositeAccount.slug,oppositeAccount.isIncognito,oppositeAccount.type,oppositeAccount.__typename,permissions.id,permissions.canRefund,permissions.canDownloadInvoice,permissions.canReject,permissions.__typename,paymentMethod.name,paymentMethod.service,paymentMethod.sourcePaymentMethod,paymentMethod.type,expense.id,expense.type,expense.description,expense.invoiceInfo,expense.tags,expense.payee.id,expense.payee.name,expense.payee.slug,expense.payee.type,expense.payee.__typename,expense.__typename,refundTransaction.id,giftCardEmitterAccount.id,giftCardEmitterAccount.name,giftCardEmitterAccount.slug,giftCardEmitterAccount.__typename
8,e4e98242-b07c-426b-89af-f0474b57c4a0,e4e98242-b07c-426b-89af-f0474b57c4a0,CONTRIBUTION,NaN,NaN,c8424e2d-a5e6-4296-af70-efffe7808061,CREDIT,Monthly financial contribution to Jellyfin,2023-01-31T21:56:29.823Z,None,False,False,NaN,False,NaN,Credit,USD,500,Amount,USD,500,Amount,USD,450,Amount,USD,450,Amount,USD,-50,Amount,rmvrwng4-kj03dpbm-ddzqz57o-yl9e8xba,26434.0,Jellyfin,jellyfin,False,COLLECTIVE,Collective,mvrwng4k-j03dpbma-339qz57o-yl9e8xba,582410,Viacheslav Ostroukh,slavoutich,False,INDIVIDUAL,Individual,a47byg9n-xozdp8we-vjrpmjlv-03rek5w8,False,False,False,TransactionPermissions,None,STRIPE,NaN,CREDITCARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,0e2e90fd-cef4-4858-a025-eed87b5eb2a9,0e2e90fd-cef4-4858-a025-eed87b5eb2a9,CONTRIBUTION,NaN,NaN,95d5d5e8-3b76-4a5c-a56a-9cd1d812e7a4,CREDIT,Financial contribution to Buttercup,2023-01-31T21:37:07.078Z,None,False,False,NaN,False,NaN,Credit,USD,500,Amount,USD,500,Amount,USD,450,Amount,USD,450,Amount,USD,-50,Amount,eeng0kzd-yvor4pzl-5g6bma83-7xlw95jl,9173.0,Buttercup,buttercup,False,COLLECTIVE,Collective,gm9bnk80-437xqryr-4xepvzeo-ljdayw5r,604656,Jos,jos2,False,INDIVIDUAL,Individual,v349mrwg-z75lpyky-e086a08d-jeybknox,False,False,False,TransactionPermissions,None,STRIPE,NaN,CREDITCARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,6b879b26-8997-43ac-b913-b3d6c2beabc5,6b879b26-8997-43ac-b913-b3d6c2beabc5,CONTRIBUTION,NaN,NaN,fc61dd61-4288-4056-b4b9-bb78ae66421b,CREDIT,Monthly financial contribution to BootstrapVue...,2023-01-31T21:02:13.449Z,None,False,False,NaN,False,NaN,Credit,USD,2500,Amount,USD,2500,Amount,USD,2372,Amount,USD,2372,Amount,USD,-128,Amount,dgm9bnk8-0437xqr4-wepvzeol-jdayw5re,10056.0,BootstrapVue,bootstrap-vue,False,COLLECTIVE,Collective,a47byg9n-xozdp8y9-azmqmjlv-03rek5w8,558276,casino seo,casino-seo,False,INDIVIDUAL,Individual,mywxoz34-09rl6k9e-nlyqvenb-dj7gk85a,False,False,False,TransactionPermissions,None,STRIPE,NaN,CREDITCARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,738ac263-7aab-4817-9dc8-035b1859f4d8,738ac263-7aab-4817-9dc8-035b1859f4d8,CONTRIBUTION,NaN,NaN,cde380ad-9bef-4319-a982-4dcb64f094b0,CREDIT,Monthly financial contribution to Logseq (Back...,2023-01-31T20:55:27.881Z,None,False,False,NaN,False,NaN,Credit,USD,500,Amount,USD,500,Amount,USD,450,Amount,USD,450,Amount,USD,-50,Amount,lk9mbw7y-48r3zq3m-gddp0ej5-lavnodgx,166914.0,Logseq,logseq,False,COLLECTIVE,Collective,mvrwng4k-j03dpbmn-ov9qz57o-yl9e8xba,604010,Detlev Bruggmann,detlev-bruggmann,False,INDIVIDUAL,Individual,gm9bnk80-437xqrw4-mke6vzeo-ljdayw5r,False,False,False,Transa

In [101]:
# for each row in df_stripe, find the corresponding row in df_platform, match df_stripe['id'] with df_platform['merchantId']
# if the row exists, add the platform transaction id to df_stripe in a new column called platform_transaction_id
# if the row does not exist, set the platform_transaction_id to None

# Merge the two dataframes on the 'id' from df_stripe and 'merchantId' from df_platform
merged_df = pd.merge(df_s, df_platform[['merchantId', 'id']], left_on='id', right_on='merchantId', how='left', suffixes=('', '_platform_transaction'))

# Drop the extra 'merchantId' column from the merge
merged_df.drop(columns='merchantId', inplace=True)

# Assign the resulting dataframe back to df_stripe
df_s = merged_df

In [102]:
# count the number of transactions that have a id_platform_transaction
df_s['id_platform_transaction'].count()

0